Відкриття гугл диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import os.path
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torchvision import transforms
import math
import torch.nn as nn
import torch.nn.functional as F
import time

Дата лоадер

In [ ]:
def get_pathes(base_folder ):
    paths = []
    for i in range(11):
        folder_name = f'{i:02d}'
        image_folder = os.path.join(base_folder, folder_name, 'image_0')
        movements_file = os.path.join(base_folder, folder_name, f'{folder_name}.txt')
        paths.append((image_folder,movements_file))
    return paths

def get_data(batch_size, base_folder = '/content/drive/MyDrive'):
    paths = get_pathes(base_folder)

    # Вибір папок для тренування, валідації і тестування
    validation_folder = paths[-2]  # Предостання папка
    test_folder = paths[-1]        # Остання папка
    train_folders = paths[:-2]     # Всі інші папки

    # Читання рухання з файлу
    def create_movements(movements_file):
        movements = []
        with open(movements_file, 'r') as f:
            for line in f:
                data = list(map(float, line.strip().split()))
                movements.append(data)
        return movements

    # movements = create_movements(movements_file)

    # Перетворення рухів
    # def transform_mov(movements):
    #     new_movements = []
    #     for matrix in movements:
    #         new_matrix = [
    #             matrix[0:4],
    #             matrix[4:8],
    #             matrix[8:12],
    #             [0, 0, 0, 1]
    #         ]
    #         new_matrix = torch.tensor(new_matrix, dtype=torch.float32)
    #         new_movements.append(new_matrix)

    #     res_movements = []
    #     for i in range(len(new_movements) - 1):
    #         T1_inv = torch.linalg.inv(new_movements[i])
    #         T_rel = torch.matmul(new_movements[i + 1], T1_inv)
    #         res_matrix = T_rel[:3]
    #         res_matrix = torch.cat([res_matrix[0], res_matrix[1], res_matrix[2]])
    #         res_movements.append(res_matrix)


    #     return res_movements

    # def transform_mov(movements):
    #     res_movements = []
    #     for i in range(len(movements)-1):
    #         m1 = torch.tensor(movements[i])
    #         m2 = torch.tensor(movements[i+1])
    #         res = m2-m1
    #         res_movements.append(res)
    #     return res_movements
    step = 1

    def transform_mov(movements):
        res_movements = []
        # for i in range(len(movements)-1):
        for i in range(len(movements)-step):
            m1 = torch.tensor(movements[i])
            m2 = torch.tensor(movements[i+step])
            res = m2-m1
            distance = math.sqrt(pow(res[3],2) + pow(res[7],2) + pow(res[11],2))
            res_movements.append([distance ])
        return res_movements



    # movements = transform_mov(movements)

    # класс Dataset
    class ImageDataset(Dataset):
        def __init__(self, image_folder, movements, transform=None ):
            # Отримуємо список всіх файлів .png і сортуємо їх
            all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])
            self.image_files = all_image_files
            self.movements = movements
            self.transform = transform

        def __len__(self):
            # Повертаємо мінімальну довжину між рухами та зображеннями - 1
            return min(len(self.movements), len(self.image_files) - 1)


        def __getitem__(self, idx):
            image1_path = self.image_files[idx]
            # image2_path = self.image_files[idx + 1]
            image2_path = self.image_files[idx + step]

            image1 = Image.open(image1_path).convert('L')
            image2 = Image.open(image2_path).convert('L')

            if self.transform:
                image1 = self.transform(image1)
                image2 = self.transform(image2)

            movement = self.movements[idx].clone().detach().float()if isinstance(self.movements[idx], torch.Tensor) else torch.tensor(self.movements[idx], dtype=torch.float32)
            return (image1, image2), movement


    # Трансформації для зменшення використання пам'яті
    transform = transforms.Compose([
        transforms.Grayscale(),
        # transforms.Resize((376,1240)),
        transforms.CenterCrop((370,1226)),
        # transforms.Resize((180,320)),
        transforms.ToTensor()
    ])

    def load_dataset(folders):
        images, movements = [], []
        for image_folder, movements_file in folders:
            mov = create_movements(movements_file)
            mov = transform_mov(mov)
            movements.append(mov)
            images.append(image_folder)
        return images, movements

    train_images, train_movements = load_dataset(train_folders)
    train_dataset = []
    for index,image_folder in enumerate(train_images):
        dataset = ImageDataset(image_folder, train_movements[index], transform=transform)
        train_dataset.append(dataset)
    train_dataset = torch.utils.data.ConcatDataset(train_dataset)

    # Завантаження валідаційних даних
    val_images, val_movements = load_dataset([validation_folder])
    val_dataset = ImageDataset(val_images[0], val_movements[0], transform=transform)


   # Завантаження тестових даних
    test_images, test_movements = load_dataset([test_folder])
    test_dataset = ImageDataset(test_images[0], test_movements[0], transform=transform)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=4, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)

    return train_loader, validation_loader, test_loader


EVALUATION

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def compute_accuracy(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    correct_predictions = 0   # Лічильник коректних передбачень
    total_samples = 0         # Загальна кількість елементів

    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for batch_idx, ((feature1, feature2),targets) in enumerate(data_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1, feature2)
            # print(predicted_y[0].item(), '   ', targets[0].item())

            # Обчислення помилки
            loss = criterion(predicted_y, targets)
            total_loss += loss.item()
            num_batches += 1

            # Перевірка критерію якості
            relative_difference = torch.abs((predicted_y - targets))  # Розрахунок абсолютного відхилення
            valid_predictions = torch.all(relative_difference <= 0.1, dim=1)    # Умова: відхилення <= 10 см
            correct_predictions += torch.sum(valid_predictions).item()
            total_samples = total_samples + targets.size(0)


    avg_loss = total_loss / num_batches             # Розрахунок середньої помилки
    accuracy = correct_predictions / total_samples  # Розрахунок частки вірних передбачень
    return avg_loss, accuracy

Model

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = (7,7), stride = (2,2), padding=(3, 3)),# 185, 613
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),

            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 185, 613
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),

            nn.Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),#93, 307
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 93, 307
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),#47, 154
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),

            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#47, 154
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),

            nn.Conv2d(128,256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),# 24, 77
            nn.BatchNorm2d(256),
            nn.ReLU(inplace = True),

            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 24, 77
            nn.BatchNorm2d(256),
            nn.ReLU(inplace = True),

            nn.Conv2d(256,512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),# 12, 39
            nn.BatchNorm2d(512),
            nn.ReLU(inplace = True),

            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 12, 39
            nn.BatchNorm2d(512),
            nn.ReLU(inplace = True),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),  #  6 × 20
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),  #  3 × 10
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            # nn.AdaptiveAvgPool2d((5, 5))  # Глобальний пулінг до фіксованого розміру
        )
        self.lstm1 = nn.LSTMCell(input_size = 512 * 3 * 10, hidden_size=128)
        self.lstm2 = nn.LSTMCell(input_size = 512 * 3 * 10, hidden_size=128)

        self.network = nn.Sequential(
            nn.Linear(128, 1)
        )

    def forward(self,image1,image2):
        out_features1 = self.features(image1)
        out_features2 = self.features(image2)
        out_features1 = torch.flatten(out_features1, 1)
        out_features2 = torch.flatten(out_features2, 1)
        batch_size = out_features1.size(0)

        # Ініціалізуємо приховані стани та стани осередків для lstm1
        h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)
        c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)

        # Пропускаємо ознаки першої картинки через lstm1 (один часовий крок)
        h_t1_next, c_t1_next = self.lstm1(out_features1, (h_t1, c_t1))
        h_t2_next, c_t2_next = self.lstm2(out_features2, (h_t1_next, c_t1_next))
        out = self.network(h_t2_next) # [batch_size, 1]
        return out


    # def forward(self,image1,image2):
    #     images = torch.cat((image1,image2),dim = 1)

    #     out_features = self.features(images)
    #     out_features = torch.flatten(out_features, 1)
    #     batch_size = out_features.size(0)

    #     # Ініціалізуємо приховані стани та стани осередків для lstm1
    #     h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)
    #     c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)

    #     # Пропускаємо ознаки першої картинки через lstm1 (один часовий крок)
    #     h_t1_next, c_t1_next = self.lstm1(out_features, (h_t1, c_t1))

    #    # Пропускаємо ознаки першої картинки через lstm2 (один часовий крок)
    #     h_t2_next, c_t2_next = self.lstm2(h_t1_next, (h_t1_next, c_t1_next))

    #     out = self.network(h_t2_next) # [batch_size, 1]
    #     return out

Train

In [ ]:
def train(model, num_epochs, last_epoch, train_loader,
                validation_loader, test_loader,optimizer, device,scheduler,
          checkpoint_dict):
    start_time = time.time()
    minibatch_loss_list, train_loss_list, valid_loss_list = [] ,[], []
    criterion = torch.nn.MSELoss()

    for epoch in range(last_epoch, num_epochs):
        model.train()
        for batch_idx,((feature1,feature2),targets) in enumerate(train_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1,feature2)
            loss = criterion(predicted_y, targets)

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            minibatch_loss_list.append(loss.item())
            if batch_idx %100 == 0:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}'
                      f'| Learning rate: {optimizer.param_groups[0]["lr"]}')
        # scheduler.step()
        model.eval()
        with torch.no_grad():
            avg_loss_train, accuracy_train = compute_accuracy(model, train_loader, device)
            avg_loss_val, accuracy_val = compute_accuracy(model, validation_loader, device)
            # scheduler.step( avg_loss_val)
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train AVG LOSS: {avg_loss_train: .4f} | Accuracy_train: {accuracy_train: .4f} \n'
                  f'| Validation AVG LOSS: {avg_loss_val: .4f} | Accuracy_val: {accuracy_val: .4f} ')
            train_loss_list.append(avg_loss_train)
            valid_loss_list.append(avg_loss_val)

        scheduler.step()

        # checkpoint = {
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict()
        # }

        checkpoint_dict['state_model'] = model.state_dict()
        checkpoint_dict['state_opt'] =  optimizer.state_dict()
        checkpoint_dict[ 'state_scheduler'] = scheduler.state_dict()
        checkpoint_dict['train_loss'] = avg_loss_train
        checkpoint_dict['val_loss'] = avg_loss_val
        checkpoint_dict['train_acc'] = accuracy_train
        checkpoint_dict['val_acc'] =  accuracy_val
        checkpoint_dict['EPOCHS'] = num_epochs
        checkpoint_dict['current_epoch'] = epoch + 1
        checkpoint_dict[ 'learning'] = optimizer.param_groups[0]["lr"]

        path = directory_path + str(epoch + 1) + '_epoch.pth'
        torch.save(checkpoint_dict, path)
        # path = '/content/drive/My Drive/checkpoint_model_3.pth'
        # torch.save(checkpoint, path)

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    avg_loss_test, accuracy_test = compute_accuracy(model, test_loader,device)
    print(f'Test AVG LOSS: {avg_loss_test: .4f} | Accuracy_test: {accuracy_test: .4f}')

    return minibatch_loss_list, train_loss_list, valid_loss_list


Опис моделі, оптимізатора, шедулера

In [ ]:
txt_model_opt_shed = '''
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size = (7,7), stride = (2,2), padding=(3, 3)),# 185, 613
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),

            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 185, 613
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),

            nn.Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),#93, 307
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 93, 307
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),#47, 154
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),

            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#47, 154
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),

            nn.Conv2d(128,256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),# 24, 77
            nn.BatchNorm2d(256),
            nn.ReLU(inplace = True),

            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 24, 77
            nn.BatchNorm2d(256),
            nn.ReLU(inplace = True),

            nn.Conv2d(256,512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),# 12, 39
            nn.BatchNorm2d(512),
            nn.ReLU(inplace = True),

            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),# 12, 39
            nn.BatchNorm2d(512),
            nn.ReLU(inplace = True),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),  #  6 × 20
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),  #  3 × 10
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            # nn.AdaptiveAvgPool2d((5, 5))  # Глобальний пулінг до фіксованого розміру
        )
        self.lstm1 = nn.LSTMCell(input_size = 512 * 3 * 10, hidden_size=128)
        self.lstm2 = nn.LSTMCell(input_size = 128, hidden_size=128)

        self.network = nn.Sequential(
            nn.Linear(128, 1)
        )

    # def forward(self,image1,image2):
    #     out_features1 = self.features(image1)
    #     out_features2 = self.features(image2)
    #     out_features1 = torch.flatten(out_features1, 1)
    #     out_features2 = torch.flatten(out_features2, 1)
    #     batch_size = out_features1.size(0)

    #     # Ініціалізуємо приховані стани та стани осередків для lstm1
    #     h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)
    #     c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)

    #     # Пропускаємо ознаки першої картинки через lstm1 (один часовий крок)
    #     h_t1_next, c_t1_next = self.lstm1(out_features1, (h_t1, c_t1))
    #     h_t2_next, c_t2_next = self.lstm2(out_features2, (h_t1_next, c_t1_next))
    #     out = self.network(h_t2_next) # [batch_size, 1]
    #     return out


    def forward(self,image1,image2):
        images = torch.cat((image1,image2),dim = 1)

        out_features = self.features(images)
        out_features = torch.flatten(out_features, 1)
        batch_size = out_features.size(0)

        # Ініціалізуємо приховані стани та стани осередків для lstm1
        h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)
        c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)

        # Пропускаємо ознаки першої картинки через lstm1 (один часовий крок)
        h_t1_next, c_t1_next = self.lstm1(out_features, (h_t1, c_t1))

       # Пропускаємо ознаки першої картинки через lstm2 (один часовий крок)
        h_t2_next, c_t2_next = self.lstm2(h_t1_next, (h_t1_next, c_t1_next))

        out = self.network(h_t2_next) # [batch_size, 1]
        return out
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0003)

'''

Создание библиотеки для хранения сохранения

In [ ]:
directory_path = '/content/drive/My Drive/checpoint/bestself_lstm_1ch/'

# Перевірка існування директорії
if not os.path.exists(directory_path):
    # Створення директорії
    os.makedirs(directory_path)
    print(f"Директорія {directory_path} була створенна.")
else:
    print(f"Директорія {directory_path} вже існує.")

Директорія /content/drive/My Drive/checpoint/bestself_lstm_1ch/ була створенна.


Створення словника для збереження результатів

In [ ]:
checkpoint_dict = {
    'model_description' : txt_model_opt_shed,
    'state_model' : None,
    'state_opt'   : None,
    'state_scheduler' : None,

    'train_loss' :None,
    'val_loss'   : None,
    'best_loss'  : None,
    'train_acc'  : None,
    'val_acc'    : None,

    'EPOCHS'     : None,
    'current_epoch' : None,
    'learning' : None
}

MAIN

In [ ]:
BATCH_SIZE = 8
RANDOM_SEED = 123
NUM_EPOCHS = 5
last_epoch = 0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Використовується пристрій: {DEVICE}")

Використовується пристрій: cuda


In [ ]:
from torch.optim.lr_scheduler import StepLR
set_all_seeds(RANDOM_SEED)

model = Network()
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0003)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
train_loader, validation_loader, test_loader = get_data(batch_size = BATCH_SIZE)
# scheduler = StepLR(optimizer, step_size = 4, gamma = 0.3)
scheduler = None
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,464,289 trainable parameters


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Завантаження словника з файлу
path = './2_epoch.pth'
checkpoint = torch.load(path)
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
# # Завантаження стану моделі
# save_check = checkpoint['optimizer_state_dict']
# save_check['lr'] = 0.0003
model.load_state_dict( checkpoint['state_model'] )
optimizer.load_state_dict( checkpoint['state_opt'] )
scheduler.load_state_dict( checkpoint['state_scheduler'] )
last_epoch = checkpoint['current_epoch']
# # Завантаження стану оптимізатора
# optimizer.load_state_dict(save_check)

In [ ]:
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = NUM_EPOCHS,
            last_epoch = last_epoch,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler,
            checkpoint_dict = checkpoint_dict)

Epoch: 001/005 | Batch 0000/2550 | Loss: 0.7455| Learning rate: 0.0001
Epoch: 001/005 | Batch 0100/2550 | Loss: 0.1454| Learning rate: 0.0001
Epoch: 001/005 | Batch 0200/2550 | Loss: 0.2166| Learning rate: 0.0001
Epoch: 001/005 | Batch 0300/2550 | Loss: 0.0735| Learning rate: 0.0001
Epoch: 001/005 | Batch 0400/2550 | Loss: 0.0152| Learning rate: 0.0001
Epoch: 001/005 | Batch 0500/2550 | Loss: 0.0451| Learning rate: 0.0001
Epoch: 001/005 | Batch 0600/2550 | Loss: 0.0705| Learning rate: 0.0001
Epoch: 001/005 | Batch 0700/2550 | Loss: 0.0245| Learning rate: 0.0001
Epoch: 001/005 | Batch 0800/2550 | Loss: 0.0470| Learning rate: 0.0001
Epoch: 001/005 | Batch 0900/2550 | Loss: 0.1075| Learning rate: 0.0001
Epoch: 001/005 | Batch 1000/2550 | Loss: 0.0342| Learning rate: 0.0001
Epoch: 001/005 | Batch 1100/2550 | Loss: 0.0073| Learning rate: 0.0001
Epoch: 001/005 | Batch 1200/2550 | Loss: 0.0202| Learning rate: 0.0001
Epoch: 001/005 | Batch 1300/2550 | Loss: 0.0143| Learning rate: 0.0001
Epoch:

In [ ]:
avg_loss_val, accuracy_val = compute_accuracy(model, validation_loader, DEVICE)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


0.44122523069381714     0.2889880836009979
0.6314010620117188     0.3602379560470581
1.173960566520691     0.45316970348358154
1.100450038909912     0.5605986714363098
1.0535552501678467     0.6563873291015625
1.0378566980361938     0.7622151374816895
1.1572362184524536     0.843492865562439
1.2384068965911865     0.928745448589325
1.232290506362915     1.0433317422866821
1.310836911201477     1.076290249824524
1.171535611152649     1.0711132287979126
1.077881932258606     1.082194447517395
1.2542418241500854     1.113807201385498
1.2371474504470825     1.151640772819519
1.1208637952804565     1.1913796663284302
1.1752381324768066     1.2265307903289795
1.1012141704559326     1.250442624092102
1.0784406661987305     1.2774924039840698
1.1523863077163696     1.2363101243972778
1.0809382200241089     1.1399396657943726
0.7766743302345276     1.1011850833892822
0.979907214641571     1.0649473667144775
1.5948611497879028     0.9946592450141907
1.070954442024231     0.91362065076828
1.00876